### Dependencies

In [ ]:
import os
from dataclasses import dataclass
from pathlib import Path

import tensorflow as tf
from keras.layers import (Concatenate, Dense, Dropout, Embedding, Flatten,
                          Input, Multiply)
from keras.models import Model

from recommender_system.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from recommender_system.logging import logger
from recommender_system.utils import create_directories, read_yaml

os.chdir("../")

### Entity

In [ ]:
@dataclass(frozen=True)
class ModelConfig:
    """Represents the configuration for building the model."""
    root_dir: Path
    model_path: Path
    number_of_reviewers: int
    number_of_products: int
    number_of_dimensions: int
    learning_rate: float

### Configuration Manager

In [ ]:
class ConfigurationManager:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):
        """Initialises ConfigurationManager with config and params filepaths."""
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_build_model_config(self) -> ModelConfig:
        """Returns the model building configuration."""
        config = self.config.build_model
        create_directories([config.root_dir])

        build_model_config = ModelConfig(
            root_dir=Path(config.root_dir),
            model_path=Path(config.model_path),
            number_of_reviewers=self.params.NUMBER_OF_REVIEWERS,
            number_of_products=self.params.NUMBER_OF_PRODUCTS,
            number_of_dimensions=self.params.NUMBER_OF_DIMENSIONS,
            learning_rate=self.params.LEARNING_RATE
        )

        return build_model_config

### Component

In [ ]:
class ModelBuilder:
    def __init__(self, config):
        """Initialises the ModelBuilder object with the given config."""
        self.config = config

    def build_and_compile_model(self):
        """Builds and compiles the model."""
        number_of_reviewers = self.config.number_of_reviewers
        number_of_products = self.config.number_of_products
        number_of_dimensions = self.config.number_of_dimensions
        learning_rate = self.config.learning_rate

        reviewer_input = Input(shape=(1,))
        reviewer_embedding = Embedding(
            input_dim=number_of_reviewers, output_dim=number_of_dimensions
        )(reviewer_input)
        reviewer_embedding = Flatten()(reviewer_embedding)

        product_input = Input(shape=(1,))
        product_embedding = Embedding(
            input_dim=number_of_products, output_dim=number_of_dimensions
        )(product_input)
        product_embedding = Flatten()(product_embedding)

        matrix_factorisation_output = Multiply()(
            [reviewer_embedding, product_embedding]
        )
        concatenated_embeddings = Concatenate()([reviewer_embedding, product_embedding])

        hidden_layer_1 = Dense(units=32, activation="relu")(concatenated_embeddings)
        dropout_layer = Dropout(rate=0.5)(hidden_layer_1)
        hidden_layer_2 = Dense(units=16, activation="relu")(dropout_layer)

        output = Concatenate()([matrix_factorisation_output, hidden_layer_2])
        output = Dense(units=1, activation="sigmoid")(output)

        model = Model(inputs=[reviewer_input, product_input], outputs=output)
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss="mse",
            metrics=["accuracy"]
        )

        model.summary()
        return model

    def save_model(self, model):
        """Saves the model."""
        model.save(self.config.model_path)

### Pipeline

In [ ]:
try:
    config = ConfigurationManager()
    model_config = config.get_build_model_config()
    model_builder = ModelBuilder(model_config)
    model = model_builder.build_and_compile_model()
    model_builder.save_model(model)
except Exception as e:
    raise e